In [ ]:
def reflect_on_image_and_regenerate(
    chart_path: str,
    instruction: str,
    model_name: str,
    out_path_v2: str,
    code_v1: str,
) -> tuple[str, str]:
    """ 
    Critique the chart IMAGE and the original code against the instruction, 
    then return refines matplotlib code.
    Returns (feedback, refine_code_with_tags).
    Supports OpenAI and Anthropic (Claude).
    """
    media_type, b64 = utils.encode_image_b64(chart_path)

    prompt = f"""
You are a data visualization expert.
Your task : critique the attached chart and the original code against the given instruction, 
then return the improved matplotlib code.

Original code (for context):
{code_v1}

OUTPUT FORMAT (STRICT):
1) First line: a valid JSON object with ONLY the \"feedback\" field.
Example:{{\"feedback\": \"The legend is unclear and the axis labels overlap.\"}}

2) After a newline, output ONLY the refined Python code wrapped in:
<execute_python>
...
</execute_python>

3) Import all necessary libraries in the code. Don't assume any imports from the original code.

HARD CONSTRAINTS:
- Do NOT include Markdown, backticks, or any extra prose outside the two parts above.
- Use pandas/matplotlib only (no seaborn).
- Assume df already exists; do not read from files.
- Save to '{out_path_v2}' with dpi=300.
- Always call plt.close() at the end (no plt.show()).
- Include all necessary import statements.

Schema (columns available in df):
- date (M/D/YY)
- time (HH:MM)
- cash_type (card or cash)
- card (string)
- price (number)
- coffee_name (string)
- quarter (1-4)
- month (1-12)
- year (YYYY)

Instruction: 
{instruction}
"""

    # In case the name is "Claude" or "Anthropic", use the safe helper
    lower = model_name.lower()
    if "claude" in lower or "anthropic" in lower:
        # Use safe helper that joins all text blocks and adds a system prompt
        content = utils.image_anthropic_call(model_name, prompt, media_type, b64)
    else: 
        content = utils.image_openai_call(model_name, prompt, media_type, b64)

    # Parse ONLY the first JSON line (feedback)
    lines = content.splitlines()
    json_line = lines[0].strip() if lines else ""

    try:
        obj = json.loads(json_line)
    except Exception as e:
        # Fallback: try to capture the first {...} in all the content
        m_json = re.search(r"\{.*?\}", content, flags=re.DOTALL)
        if m_json:
            try:
                obj = json.loads(m_json.group(0))
            except Exception as e2:
                obj = {"feedback": f"Failed to parse JSON: {e2}", "refined_code": ""}
        else:
            obj = {"feedback": f"Failed to find JSON: {e}", "refined_code": ""}

    # Extract refined code from <execute_python>...</execute_python>
    m_code = re.search(r"<execute_python>([\s\S]*?)</execute_python>", content)
    refined_code_body = m_code.group(1).strip() if m_code else ""
    refined_code = utils.ensure_execute_python_tags(refined_code_body)

    feedback = str(obj.get("feedback", "")).strip()
    return feedback, refined_code